In [52]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import skimage
from tqdm import tqdm

from IPython.display import clear_output

In [53]:
work_dir = "./raw_data/"
train_data_dir = "train_data/"
tile_dir = work_dir + "tiles/"
mask_dir = work_dir + "masks/"

tile_paths = sorted([tile_dir + f for f in os.listdir(tile_dir)])
mask_paths = sorted([mask_dir + f for f in os.listdir(mask_dir)])

In [54]:
def parse_number_str(s):
    return ''.join(filter(str.isdigit, s))

In [ ]:
from src.Segmentation import label_image

os.makedirs(work_dir + "separated_masks", exist_ok=True)

for mask_path in mask_paths:
    mask = Image.open(mask_path)
    mask = np.array(mask)

    separated_masks = label_image(mask)
    separated_masks = separated_masks / separated_masks.max() * 255
    separated_masks = separated_masks.astype(np.uint8)
    np.save(work_dir + "separated_masks/" + parse_number_str(mask_path), separated_masks)

    # demonstration purposes
    clear_output(wait=True)
    plt.title("Original mask and separated masks")
    plt.axis('off')
    plt.subplot(1, 2, 1)
    plt.imshow(separated_masks)
    plt.axis('off')
    plt.subplot(1, 2, 2)
    plt.imshow(mask)
    plt.axis('off')
    plt.show()
clear_output(wait=True)

---

```
<class-index> <x1> <y1> <x2> <y2> ... <xn> <yn>
```

In [56]:
separated_mask_paths = sorted([work_dir + "separated_masks/" + f for f in os.listdir(work_dir + "separated_masks")])

os.makedirs(train_data_dir + "separated_mask_edges", exist_ok=True)

for separated_mask_path in tqdm(separated_mask_paths):
    separated_mask = np.load(separated_mask_path)
    
    unique_values = np.unique(separated_mask)

    yolo_input = ""
    
    for value in unique_values:
        if value == 0: continue # background

        mask = np.zeros_like(separated_mask)
        mask[separated_mask == value] = 255

        # get the bounding coordinates of the mask
        mask_edges = np.zeros_like(mask)
        mask_edges += mask - mask[[-1, *list(range(0, len(mask)-1))], :]
        mask_edges += mask - mask[[*list(range(1, len(mask))), 0], :]
        mask_edges += mask - mask[:, [-1, *list(range(0, len(mask)-1))]]
        mask_edges += mask - mask[:, [*list(range(1, len(mask))), 0]]
        mask_edges = mask_edges > 0
        mask_edges = mask_edges.astype(np.uint8) * 255

        x, y = np.where(mask_edges)
        x = x.astype(str)
        y = y.astype(str)
        
        yolo_input += "0 " + " ".join([" ".join(pair) for pair in zip(x, y)]) + "\n"

    with open(train_data_dir + "separated_mask_edges/" + parse_number_str(separated_mask_path) + ".txt", "w") as f:
        f.write(yolo_input)
        f.close()

100%|██████████| 38/38 [00:18<00:00,  2.08it/s]


---

In [50]:
from ultralytics import YOLO

model = YOLO('model/raw_YOLOv8n_segmentation.pt') 

/Users/nomomon/Desktop/Projects/DLF-Single-Tree-Detection/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
